In [2]:
import psycopg2
import pandas as pd 
import os

In [3]:
conn = psycopg2.connect(host ="localhost", database="mydb",user="postgres",password="1234567890#Aa")
df = pd.read_sql("SELECT * FROM customers", conn)

C:\Users\Dell\AppData\Local\Temp\ipykernel_8748\3933280288.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM customers", conn)


In [4]:
df

,id,name,email
0,1,Alice,alice@example.com
1,2,rahul,rahul@example.com
2,3,rahulG,rahulG@example.com
3,4,boss,boss@example.com
4,5,badmass,badmass@example.com
5,6,kali,kali@example.com
6,7,dogesh,dogesh@example.com
7,8,billu,billu@example.com
8,9,jivi,jivi@example.com
9,10,muska,muska@example.com


In [5]:
import json
# rows = json.loads(df.to_json(orient="records",date_format="iso"))

rows = df.where(pd.notnull(df), None).to_dict(orient="records")
rows = json.dumps(rows, default=str)

In [6]:
rows

'[{"id": 1, "name": "Alice", "email": "alice@example.com"}, {"id": 2, "name": "rahul", "email": "rahul@example.com"}, {"id": 3, "name": "rahulG", "email": "rahulG@example.com"}, {"id": 4, "name": "boss", "email": "boss@example.com"}, {"id": 5, "name": "badmass", "email": "badmass@example.com"}, {"id": 6, "name": "kali", "email": "kali@example.com"}, {"id": 7, "name": "dogesh", "email": "dogesh@example.com"}, {"id": 8, "name": "billu", "email": "billu@example.com"}, {"id": 9, "name": "jivi", "email": "jivi@example.com"}, {"id": 10, "name": "muska", "email": "muska@example.com"}, {"id": 11, "name": "neha", "email": "neha@example.com"}, {"id": 12, "name": "priya", "email": "priya@example.com"}, {"id": 13, "name": "neha", "email": "neha@example.com"}, {"id": 14, "name": "ridhi", "email": "ridhi@example.com"}]'

In [7]:
row = json.loads(rows)
row

[{'id': 1, 'name': 'Alice', 'email': 'alice@example.com'},
 {'id': 2, 'name': 'rahul', 'email': 'rahul@example.com'},
 {'id': 3, 'name': 'rahulG', 'email': 'rahulG@example.com'},
 {'id': 4, 'name': 'boss', 'email': 'boss@example.com'},
 {'id': 5, 'name': 'badmass', 'email': 'badmass@example.com'},
 {'id': 6, 'name': 'kali', 'email': 'kali@example.com'},
 {'id': 7, 'name': 'dogesh', 'email': 'dogesh@example.com'},
 {'id': 8, 'name': 'billu', 'email': 'billu@example.com'},
 {'id': 9, 'name': 'jivi', 'email': 'jivi@example.com'},
 {'id': 10, 'name': 'muska', 'email': 'muska@example.com'},
 {'id': 11, 'name': 'neha', 'email': 'neha@example.com'},
 {'id': 12, 'name': 'priya', 'email': 'priya@example.com'},
 {'id': 13, 'name': 'neha', 'email': 'neha@example.com'},
 {'id': 14, 'name': 'ridhi', 'email': 'ridhi@example.com'}]

In [8]:
df = pd.DataFrame(rows)
df.columns

ValueError: DataFrame constructor not properly called!

In [ ]:
required_columns = ['id', 'name', 'email']
timezone = "UTC"
for col in df.columns:
        if pd.api.types.is_object_dtype(df[col]):
            try:
                dt_series = pd.to_datetime(df[col], format="%Y-%m-%d %H:%M:%S", errors="coerce")

                if dt_series.notna().any():
                # localize naive timestamps as UTC by default, then format as ISO
                    df[col] = dt_series.dt.tz_localize(None).dt.strftime("%Y-%m-%d %H:%M:%S")
            except Exception:
                            # ignore conversion failures; keep original values
                pass

    

{"rows": [{"id": 1, "name": "Alice", "email": "alice@example.com"}, {"id": 2, "name": "rahul", "email": "rahul@example.com"}, {"id": 3, "name": "rahulG", "email": "rahulG@example.com"}, {"id": 4, "name": "boss", "email": "boss@example.com"}, {"id": 5, "name": "badmass", "email": "badmass@example.com"}, {"id": 6, "name": "kali", "email": "kali@example.com"}, {"id": 7, "name": "dogesh", "email": "dogesh@example.com"}, {"id": 8, "name": "billu", "email": "billu@example.com"}, {"id": 9, "name": "jivi", "email": "jivi@example.com"}, {"id": 10, "name": "muska", "email": "muska@example.com"}, {"id": 11, "name": "neha", "email": "neha@example.com"}, {"id": 12, "name": "priya", "email": "priya@example.com"}, {"id": 13, "name": "neha", "email": "neha@example.com"}, {"id": 14, "name": "ridhi", "email": "ridhi@example.com"}], "schema": {"id": "INT64", "name": "STRING", "email": "STRING"}}


In [ ]:
transformed_rows = []
for _, row in df.iterrows():
        out = {}
        for col in required_columns:
            val = row.get(col, None) if col in df.columns else None
            if pd.isna(val):
                val = None
            if isinstance(val, (list, dict)):
                val = json.dumps(val)
            out[col] = val
        transformed_rows.append(out)



In [ ]:
schema_map = {}
for col in required_columns:
    if col not in df.columns:
        schema_map[col] = "STRING"
        continue
    dtype = df[col].dtype
    if pd.api.types.is_integer_dtype(dtype):
        schema_map[col] = "INT64"
    elif pd.api.types.is_float_dtype(dtype):
        schema_map[col] = "FLOAT64"
    elif pd.api.types.is_bool_dtype(dtype):
        schema_map[col] = "BOOL"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        schema_map[col] = "TIMESTAMP"
    else:
            
        sample = df[col].dropna().astype(str).head(10).tolist()
        is_json = True
        for s in sample:
            try:
                json.loads(s)
            except Exception:
                is_json = False
                break
        schema_map[col] = "JSON" if is_json and len(sample) > 0 else "STRING"




In [ ]:
payload = {"rows": transformed_rows, "schema": schema_map}
try:
    print(json.dumps(payload, default=str))
except Exception as e:
    print(f" Failed to serialize transformed payload:{str(e)}")

In [10]:
obj = TransformTool()
ans = obj._run(rows,['id', 'name', 'email'],"UTC")
ans

'{"rows": [{"id": 1, "name": "Alice", "email": "alice@example.com"}, {"id": 2, "name": "rahul", "email": "rahul@example.com"}, {"id": 3, "name": "rahulG", "email": "rahulG@example.com"}, {"id": 4, "name": "boss", "email": "boss@example.com"}, {"id": 5, "name": "badmass", "email": "badmass@example.com"}, {"id": 6, "name": "kali", "email": "kali@example.com"}, {"id": 7, "name": "dogesh", "email": "dogesh@example.com"}, {"id": 8, "name": "billu", "email": "billu@example.com"}, {"id": 9, "name": "jivi", "email": "jivi@example.com"}, {"id": 10, "name": "muska", "email": "muska@example.com"}, {"id": 11, "name": "neha", "email": "neha@example.com"}, {"id": 12, "name": "priya", "email": "priya@example.com"}, {"id": 13, "name": "neha", "email": "neha@example.com"}, {"id": 14, "name": "ridhi", "email": "ridhi@example.com"}], "schema": {"id": "INT64", "name": "STRING", "email": "STRING"}}'

In [11]:
payload = json.loads(ans)
payload
# rows = payload.get("rows", [])
# rows
# schema = payload.get("schema", {})
# schema


{'rows': [{'id': 1, 'name': 'Alice', 'email': 'alice@example.com'},
  {'id': 2, 'name': 'rahul', 'email': 'rahul@example.com'},
  {'id': 3, 'name': 'rahulG', 'email': 'rahulG@example.com'},
  {'id': 4, 'name': 'boss', 'email': 'boss@example.com'},
  {'id': 5, 'name': 'badmass', 'email': 'badmass@example.com'},
  {'id': 6, 'name': 'kali', 'email': 'kali@example.com'},
  {'id': 7, 'name': 'dogesh', 'email': 'dogesh@example.com'},
  {'id': 8, 'name': 'billu', 'email': 'billu@example.com'},
  {'id': 9, 'name': 'jivi', 'email': 'jivi@example.com'},
  {'id': 10, 'name': 'muska', 'email': 'muska@example.com'},
  {'id': 11, 'name': 'neha', 'email': 'neha@example.com'},
  {'id': 12, 'name': 'priya', 'email': 'priya@example.com'},
  {'id': 13, 'name': 'neha', 'email': 'neha@example.com'},
  {'id': 14, 'name': 'ridhi', 'email': 'ridhi@example.com'}],
 'schema': {'id': 'INT64', 'name': 'STRING', 'email': 'STRING'}}

In [12]:
issues = []
for col in required_columns:
        if col not in schema:
            issues.append(f"Missing column in schema: {col}")

        # Row-level checks
for idx, row in enumerate(rows):
        for col in required_columns:
            if col not in row:
                issues.append(f"Row {idx}: missing column {col}")
            else:
                # null check for required columns
                if row[col] is None:
                    issues.append(f"Row {idx}: column {col} is null")

        # optional: require at least one non-null in required cols
for col in required_columns:
        non_null_count = sum(1 for r in rows if r.get(col) is not None)
        if non_null_count == 0:
            issues.append(f"All values are null for required column: {col}")

if issues:
            print(" Validation Failed:\n" + "\n".join(issues))
else:
      print("validation passed")

NameError: name 'required_columns' is not defined

In [68]:
ans

'{"rows": [{"id": 1, "name": "Alice", "email": "alice@example.com"}, {"id": 2, "name": "rahul", "email": "rahul@example.com"}, {"id": 3, "name": "rahulG", "email": "rahulG@example.com"}, {"id": 4, "name": "boss", "email": "boss@example.com"}, {"id": 5, "name": "badmass", "email": "badmass@example.com"}, {"id": 6, "name": "kali", "email": "kali@example.com"}, {"id": 7, "name": "dogesh", "email": "dogesh@example.com"}, {"id": 8, "name": "billu", "email": "billu@example.com"}, {"id": 9, "name": "jivi", "email": "jivi@example.com"}, {"id": 10, "name": "muska", "email": "muska@example.com"}, {"id": 11, "name": "neha", "email": "neha@example.com"}, {"id": 12, "name": "priya", "email": "priya@example.com"}, {"id": 13, "name": "neha", "email": "neha@example.com"}, {"id": 14, "name": "ridhi", "email": "ridhi@example.com"}], "schema": {"id": "INT64", "name": "STRING", "email": "STRING"}}'

In [20]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\Dell\Downloads\postgres-bigquery-agent-c25ae722d534.json"

In [24]:
from google.cloud import bigquery
from google.oauth2 import service_account

row = json.loads(ans)
# row
credential_path = r"C:\Users\Dell\Downloads\postgres-bigquery-agent-c25ae722d534.json"
credentials = service_account.Credentials.from_service_account_file(credential_path)
client =bigquery.Client(credentials=credentials, project=credentials.project_id)



In [67]:
row2= json.loads(ans).get("rows", [])
row2

[{'id': 1, 'name': 'Alice', 'email': 'alice@example.com'},
 {'id': 2, 'name': 'rahul', 'email': 'rahul@example.com'},
 {'id': 3, 'name': 'rahulG', 'email': 'rahulG@example.com'},
 {'id': 4, 'name': 'boss', 'email': 'boss@example.com'},
 {'id': 5, 'name': 'badmass', 'email': 'badmass@example.com'},
 {'id': 6, 'name': 'kali', 'email': 'kali@example.com'},
 {'id': 7, 'name': 'dogesh', 'email': 'dogesh@example.com'},
 {'id': 8, 'name': 'billu', 'email': 'billu@example.com'},
 {'id': 9, 'name': 'jivi', 'email': 'jivi@example.com'},
 {'id': 10, 'name': 'muska', 'email': 'muska@example.com'},
 {'id': 11, 'name': 'neha', 'email': 'neha@example.com'},
 {'id': 12, 'name': 'priya', 'email': 'priya@example.com'},
 {'id': 13, 'name': 'neha', 'email': 'neha@example.com'},
 {'id': 14, 'name': 'ridhi', 'email': 'ridhi@example.com'}]

In [69]:
# errors = client.insert_rows_json("postgres-bigquery-agent.cusitomerDB.customer_details" , rows)
table_id= "postgres-bigquery-agent.cusitomerDB.friend_details"

In [70]:
project, dataset, table = table_id.split(".")
dataset 
dataset_ref = client.dataset(dataset)
dataset_ref
table_ref = dataset_ref.table(table)
table_ref


TableReference(DatasetReference('postgres-bigquery-agent', 'cusitomerDB'), 'friend_details')

In [71]:
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    autodetect=True,
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
)

In [ ]:
ndjson_data ="\n".join([json.dumps(row) for row in row2])
# ndjson_data
json_byte = ndjson_data.encode("utf-8")
# json_byte
import io
load_job= client.load_table_from_file(
    file_obj=io.BytesIO(json_byte),
    destination=table_ref,
    job_config=job_config,
)
 

LoadJob<project=postgres-bigquery-agent, location=asia-south2, id=e1ee36f5-5c80-4b88-a55e-e24556c10b92>

In [ ]:
load_job.result() 

BadRequest: 400 Provided Schema does not match Table postgres-bigquery-agent:cusitomerDB.customer_details. Cannot add fields (field: email); reason: invalid, message: Provided Schema does not match Table postgres-bigquery-agent:cusitomerDB.customer_details. Cannot add fields (field: email)